In [97]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [98]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)
recession_data = data[data['Recession'] == 1]

yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
print(yearly_rec)
R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Automobile Sales during Recession Periods")
        )

# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]

    Year  Automobile_Sales
0   1980        678.950000
1   1981        633.408333
2   1982        631.150000
3   1983        589.000000
4   1990        648.981818
5   1991        655.891667
6   1992        675.400000
7   2000        607.500000
8   2001        692.133333
9   2002        676.150000
10  2008        678.144444
11  2009        631.691667
12  2020        543.575000


In [99]:
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard",style={'color': '#503D36', 'font-size': 24}),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Select Statistics',
            placeholder='Select a report type'
        ),
    ]),
    html.Div(dcc.Dropdown(id='select-year', 
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select year',
        )),
    html.Div([#TASK 2.3: Add a division for output display
    html.Div(id='output-container', className='chart-grid', style={'display':'flex'}),])
])

In [100]:
#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistic):
    if selected_statistic == 'Yearly Statistics': 
        return False
    else: 
        return True
    

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
     Input(component_id='dropdown-statistics', component_property='value')]
)
def update_output_container(selected_year, selected_statistics):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        
        # Plot 1: Automobile sales fluctuate over Recession Period (year-wise) using line chart
        # Grouping data for plotting
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Automobile Sales during Recession Periods")
        )

        # Plot 2: Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
        avg_vehicles_rec = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(avg_vehicles_rec, x='Vehicle_Type', y='Automobile_Sales', title="Average Vehicles Sold by Vehicle Type during Recessions")
        )

        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        # Grouping data for plotting
        exp_rec = recession_data.groupby('Vehicle_Type')['Total_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, values='Total_Expenditure', names='Vehicle_Type', title="Total Expenditure Share by Vehicle Type during Recessions")
        )

        # Plot 4: Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_rec = recession_data.groupby(['Vehicle_Type', 'Unemployment_Rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_rec, x='Vehicle_Type', y='Automobile_Sales', color='Unemployment_Rate', title="Effect of Unemployment Rate on Vehicle Sales during Recessions")
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)])
        ]

    # Yearly Statistic Report Plots
    elif (selected_year and selected_statistics == 'Yearly Statistic Report Plots'):
        yearly_data = data[data['Year'] == selected_year]

        # Plot 1: Yearly Automobile sales using line chart for the whole period.
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title="Yearly Automobile Sales"))

        # Plot 2: Total Monthly Automobile sales using line chart.
        monthly_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(monthly_sales, x='Month', y='Automobile_Sales', title="Total Monthly Automobile Sales"))

        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales', title=f'Average Vehicles Sold by Vehicle Type in the year {selected_year}')
        )

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        adv_exp = yearly_data.groupby('Vehicle_Type')['Advertisement_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(adv_exp, values='Advertisement_Expenditure', names='Vehicle_Type', title="Total Advertisement Expenditure by Vehicle Type")
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]

In [101]:
# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[100], line 40, in update_output_container(
    selected_year=1983,
    selected_statistics='Recession Period Statistics'
)
     34 R_chart2 = dcc.Graph(
     35     figure=px.bar(avg_vehicles_rec, x='Vehicle_Type', y='Automobile_Sales', title="Average Vehicles Sold by Vehicle Type during Recessions")
     36 )
     38 # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
     39 # Grouping data for plotting
---> 40 exp_rec = recession_data.groupby('Vehicle_Type')['Total_Expenditure'].sum().reset_index()
        recession_data =            Date  Year Month  Recession  Consumer_Confidence  \
0     1/31/1980  1980   Jan          1               108.24   
1     2/29/1980  1980   Feb          1                98.75   
2     3/31/1980  1980   Mar          1               107.48   
3     4/30/1980  1980   Ap